# Imports

In [1]:
import numpy as np
import pandas as pd
import builtins
import pyfolio as pf
# from logbook import (NestedSetup, NullHandler, Logger, StreamHandler, StderrHandler, 
#                      INFO, WARNING, DEBUG, ERROR)
from zipline.api import (order, 
                         record, 
                         symbol,
                         get_datetime,
                         order_target_percent,
                         order_target_value,
                         set_benchmark,
                         get_open_orders)
from zipline import run_algorithm
from zipline.data import bundles
from zipline.utils.calendar_utils import get_calendar
from zipline.api import order_target, record, symbol
# import pyfolio as pf
# from pyfolio.utils import extract_rets_pos_txn_from_zipline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Performance Functions

In [2]:
def rolling_sharpe(ret):
    return np.multiply(np.divide(ret.mean(), ret.std()), np.sqrt(252))

def get_rolling_stats(result, r_window):
    result["rolling_sharpe"] = result["portfolio_value"].pct_change().rolling(r_window).apply(rolling_sharpe)
    result["rolling_vol"] = result["portfolio_value"].pct_change().rolling(r_window).std()
    stats = [result.rolling_sharpe.mean(), result.portfolio_value.mean(), result.rolling_vol.mean()]
    return result, stats

def show_rolling_stats(result, r_window):
    result, stats = get_rolling_stats(result, r_window)
    fig, ax = plt.subplots(1, 3, figsize=(18,4))
    result['rolling_sharpe'].plot(ax = ax[0], title='Rolling sharpe')
    result['rolling_vol'].plot(ax = ax[1], title='Rolling vol')
    result['portfolio_value'].plot(ax = ax[2], title='Portfolio value')
    plt.show()
    return result

# Check Bundels

In [3]:
!zipline bundles

csvdir <no ingestions>
multi_asset_snp100 2023-12-17 22:09:58.629570
multi_asset_snp100 2023-12-17 22:08:14.030382
multi_asset_snp100 2023-12-17 22:07:15.246137
multi_asset_snp100 2023-12-17 22:01:09.626921
multi_asset_snp100 2023-12-17 21:56:31.607502
multi_asset_snp100 2023-12-17 21:55:36.681433
multi_asset_snp100 2023-12-17 09:13:21.717476
multi_asset_snp100 2023-12-17 09:08:47.876986
multi_asset_snp100 2023-12-17 09:06:49.789240
multi_asset_snp100 2023-12-17 08:55:08.564988
multi_asset_snp100 2023-12-17 08:52:55.696107
multi_asset_snp100 2023-12-17 08:52:18.051362
multi_asset_snp100 2023-12-17 08:48:44.729190
multi_asset_snp100 2023-12-17 08:47:54.329628
multi_asset_snp100 2023-12-16 17:31:45.414344
multi_asset_snp100 2023-12-16 15:15:33.865051
multi_asset_snp100 2023-12-16 09:20:22.560097
multi_asset_snp100 2023-12-15 22:01:14.037391
multi_asset_snp100 2023-12-15 21:54:57.685235
multi_asset_snp100 2023-12-15 21:32:50.035515
multi_asset_snp100 2023-12-15 21:28:12.594941
multi_asset

In [4]:
!zipline ingest -b multi_asset_snp100

[2023-12-18T04:11:14+0600-INFO][zipline.data.bundles.core]
 Ingesting multi_asset_snp100
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]     
[2023-12-18T04:11:15+0600-INFO][root]
 creating /home/subrina/.zipline/data/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_equities.bcolz
[2023-12-18T04:11:15+0600-INFO][root]
 copying /tmp/tmptlcal94t/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_equities.bcolz/__attrs__ -> /home/subrina/.zipline/data/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_equities.bcolz
[2023-12-18T04:11:15+0600-INFO][root]
 creating /home/subrina/.zipline/data/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_equities.bcolz/close
[2023-12-18T04:11:15+0600-INFO][root]
 copying /tmp/tmptlcal94t/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_equities.bcolz/close/__attrs__ -> /home/subrina/.zipline/data/multi_asset_snp100/2023-12-17T22;11;13.807979/daily_e

In [5]:
!zipline bundles

csvdir <no ingestions>
multi_asset_snp100 2023-12-17 22:11:13.807979
multi_asset_snp100 2023-12-17 22:09:58.629570
multi_asset_snp100 2023-12-17 22:08:14.030382
multi_asset_snp100 2023-12-17 22:07:15.246137
multi_asset_snp100 2023-12-17 22:01:09.626921
multi_asset_snp100 2023-12-17 21:56:31.607502
multi_asset_snp100 2023-12-17 21:55:36.681433
multi_asset_snp100 2023-12-17 09:13:21.717476
multi_asset_snp100 2023-12-17 09:08:47.876986
multi_asset_snp100 2023-12-17 09:06:49.789240
multi_asset_snp100 2023-12-17 08:55:08.564988
multi_asset_snp100 2023-12-17 08:52:55.696107
multi_asset_snp100 2023-12-17 08:52:18.051362
multi_asset_snp100 2023-12-17 08:48:44.729190
multi_asset_snp100 2023-12-17 08:47:54.329628
multi_asset_snp100 2023-12-16 17:31:45.414344
multi_asset_snp100 2023-12-16 15:15:33.865051
multi_asset_snp100 2023-12-16 09:20:22.560097
multi_asset_snp100 2023-12-15 22:01:14.037391
multi_asset_snp100 2023-12-15 21:54:57.685235
multi_asset_snp100 2023-12-15 21:32:50.035515
multi_asset

# Variable Initiallization for Multi Assets

In [6]:
bundle = 'multi_asset_snp100'
bundle_data = bundles.load(bundle)
sids = bundle_data.asset_finder.sids
assets = bundle_data.asset_finder.retrieve_all(sids)
# Set the start and end dates
start_date = pd.Timestamp('2011-01-01')
end_date = pd.Timestamp('2020-01-01')
capital_base = 10_000
# calendar = get_calendar('XDSE')
symb = 'AAPL'
stop_loss_percent=0.1

UnknownBundle: No bundle registered with the name 'multi_asset_snp100'

In [ ]:
print(assets)

# Zipline Functions

In [ ]:
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = bollinger_window  # Bollinger Bands window
    context.bollinger_dev = bollinger_dev  # Number of standard deviations for Bollinger Bands
    context.stop_loss_percent = stop_loss_percent

def exec_trade(data, positions):
    for asset, target_percent in positions.items():
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)

def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    for asset in context.assets:
        # Get historical price data
        prices = data.history(asset, 'close', context.bollinger_window + 1, frequency='1d')
        # print(prices)
        # Calculate Bollinger Bands
        sma = prices.mean()
        rolling_std = prices.std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)
    
        # Get the current price
        current_price = data.current(asset, 'close')
    
        open_orders = get_open_orders()
        # Check if the asset is still tradable
        if data.can_trade(asset):
            # Generate signals based on Bollinger Bands
            if current_price > upper_band:
                order_target_percent(asset, -0.5)  # Short position
                sell_signal=True
            elif current_price < lower_band:
                order_target_percent(asset, 0.5)  # Long position
                buy_signal=True
            else:
                order_target_percent(asset, 0)  # Close position
        
        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal = sell_signal,
            buy_signal = buy_signal,
            price_hist=prices
        )

In [ ]:
# Run the algorithm
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result.returns

# Volitility, Sharpe Ratio and Portfolio Value

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)

## Stop Loss

In [ ]:
# A stop price is used in conjunction with a stop order to trigger a market order when the asset's price reaches or surpasses a certain level.
# For a sell stop order, the stop price is the price at which the market order is triggered. Once the stop price is reached, the order becomes a market sell order.
# For a buy stop order, the stop price is the price at which the market order is triggered. Once the stop price is reached, the order becomes a market buy order.
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = 20  # Bollinger Bands window
    context.bollinger_dev = 1  # Number of standard deviations for Bollinger Bands
    context.stop_loss_percent = 0.1
def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    for asset in context.assets:
        prices = data.history(asset, 'close', context.bollinger_window + 1, frequency='1d')
        # Calculate Bollinger Bands
        sma = prices.mean()
        rolling_std = prices.std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)
    
        # Get the current price
        current_price = data.current(asset, 'close')
    
        open_orders = get_open_orders()
        # Check if the asset is still tradable
        if data.can_trade(asset):
            cash = context.portfolio.cash

            # Generate signals based on Bollinger Bands
            if current_price > upper_band:
                order_target(asset, -(cash // current_price),stop_price=current_price * (1 - context.stop_loss_percent))  # Short position
                sell_signal = True
            elif current_price < lower_band:
                order_target(asset, cash // current_price,stop_price=current_price * (1 + context.stop_loss_percent) )  # Long position
                buy_signal = True
            else:
                order_target(asset, 0)  # Close position

        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal=sell_signal,
            buy_signal=buy_signal,
            price_hist=prices
        )


In [ ]:
# Run the algorithm
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result.returns

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)

## Take Profit

In [ ]:
#A limit price is the specific price at which a trader wants to buy or sell an asset.
# For a buy order, the limit price is the maximum price the trader is willing to pay.
# For a sell order, the limit price is the minimum price the trader is willing to accept.
# The order will only be executed at the specified limit price or better.
take_profit_percent=0.1
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = bollinger_window  # Bollinger Bands window
    context.bollinger_dev = bollinger_dev  # Number of standard deviations for Bollinger Bands
    context.take_profit_percent = take_profit_percent  # Define Take Profit percentage

def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    take_profit_executed = False
    for asset in context.assets:
        # Get historical price data
        prices = data.history(asset, 'close', context.bollinger_window + 1, frequency='1d')

        # Calculate Bollinger Bands
        sma = prices.mean()
        rolling_std = prices.std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Get current price and open orders
        current_price = data.current(asset, 'close')
        open_orders = get_open_orders()

        # Check if asset is tradable
        if data.can_trade(asset):
            cash = context.portfolio.cash

            if current_price > upper_band:
                limit_price = current_price * (1 - context.take_profit_percent)
                order_target(asset, -(cash // current_price), limit_price=limit_price)  # Short position
            elif current_price < lower_band:
                limit_price = current_price * (1 + context.take_profit_percent)
                order_target(asset, cash // current_price, limit_price=limit_price)  

        # Record data for analysis
        record(price=current_price, upper=upper_band, lower=lower_band)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal=sell_signal,
            buy_signal=buy_signal,
            price_hist=prices,
            take_profit=take_profit_executed,
        )



In [ ]:
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)

In [ ]:
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

## Stop loss + Take profit

In [ ]:
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = 20  # Bollinger Bands window
    context.bollinger_dev = 2  # Number of standard deviations for Bollinger Bands
    context.stop_loss_percent = 0.05  # Stop loss percentage
    context.take_profit_percent = 0.1  # Take profit percentage

def handle_data(context, data):
    for asset in context.assets:
        # Get historical price data
        prices = data.history(asset, 'close', context.bollinger_window + 1, frequency='1d')
        # Calculate Bollinger Bands
        sma = prices.mean()
        rolling_std = prices.std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Get the current price
        current_price = data.current(asset, 'close')

        open_orders = get_open_orders()

        # Check if the asset is still tradable
        if data.can_trade(asset):
            # Check existing position
            position = context.portfolio.positions[asset]

            # Generate signals based on Bollinger Bands
            if current_price > upper_band:
                # Short position with stop-loss and take-profit
                order_target_percent(asset, -0.5, stop_price=current_price * (1 + context.stop_loss_percent),
                                      limit_price=current_price * (1 - context.take_profit_percent))
            elif current_price < lower_band:
                # Long position with stop-loss and take-profit
                order_target_percent(asset, 0.5, stop_price=current_price * (1 - context.stop_loss_percent),
                                      limit_price=current_price * (1 + context.take_profit_percent))
            else:
                # Close position
                order_target_percent(asset, 0)

        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            price_hist=prices
        )


# Ensemble(Bollinger Bands + MACD)

In [ ]:
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = bollinger_window  # Bollinger Bands window
    context.bollinger_dev = bollinger_dev  # Number of standard deviations for Bollinger Bands
    context.stop_loss_percent = stop_loss_percent
    context.macd_short_window = 12
    context.macd_long_window = 26
    context.macd_signal_window = 9
    context.take_profit_percent=0.3

def exec_trade(data, positions):
    for asset, target_percent in positions.items():
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)

def calculate_macd(prices, short_window, long_window, signal_window):
    exp12 = prices.ewm(span=short_window, adjust=False).mean()
    exp26 = prices.ewm(span=long_window, adjust=False).mean()
    macd = exp12 - exp26
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    
    for asset in context.assets:
        # Get historical price data
        prices = data.history(asset, 'close', context.bollinger_window + context.macd_long_window + context.macd_signal_window + 1, frequency='1d')
        # Calculate Bollinger Bands
        sma = prices[-context.bollinger_window:].mean()
        rolling_std = prices[-context.bollinger_window:].std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Calculate MACD
        close_prices = prices[-context.macd_long_window:]
        macd, signal = calculate_macd(close_prices, context.macd_short_window, context.macd_long_window, context.macd_signal_window)

        # Get the current price
        current_price = data.current(asset, 'close')
    
        open_orders = get_open_orders()
        # Check if the asset is still tradable
        if data.can_trade(asset):
            cash = context.portfolio.cash
            # Generate signals based on Bollinger Bands and MACD
            if current_price > upper_band and macd.iloc[-1] > signal.iloc[-1] and macd.iloc[-2] < signal.iloc[-2]:
                limit_price = current_price * (1 - context.take_profit_percent)
                order_target(asset, -(cash // current_price), limit_price=limit_price)# Short position
                sell_signal = True
            elif current_price < lower_band and macd.iloc[-1] < signal.iloc[-1] and macd.iloc[-2] > signal.iloc[-2]:
                limit_price = current_price * (1 + context.take_profit_percent)
                order_target(asset, cash // current_price, limit_price=limit_price)# Long position
                buy_signal = True
            else:
                order_target_percent(asset, 0)  # Close position
        
        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band, macd=macd.iloc[-1], signal=signal.iloc[-1])
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal=sell_signal,
            buy_signal=buy_signal,
            price_hist=prices
        )


In [ ]:
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)

## Added Volume Filter

In [ ]:
def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    
    for asset in context.assets:
        # Get historical price data and volume
        prices = data.history(asset, 'close', context.bollinger_window + context.macd_long_window + context.macd_signal_window + 1, frequency='1d')
        volumes = data.history(asset, 'volume', context.bollinger_window + context.macd_long_window + context.macd_signal_window + 1, frequency='1d')

        # Calculate Bollinger Bands
        sma = prices[-context.bollinger_window:].mean()
        rolling_std = prices[-context.bollinger_window:].std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Calculate MACD
        close_prices = prices[-context.macd_long_window:]
        macd, signal = calculate_macd(close_prices, context.macd_short_window, context.macd_long_window, context.macd_signal_window)

        # Get the current price and volume
        current_price = data.current(asset, 'close')
        current_volume = data.current(asset, 'volume')

        # Calculate the average volume
        average_volume = volumes.mean()

        open_orders = get_open_orders()
        # Check if the asset is still tradable
        if data.can_trade(asset):
            cash = context.portfolio.cash
            # Generate signals based on Bollinger Bands, MACD, and volume
            if (
                current_price > upper_band
                and macd.iloc[-1] > signal.iloc[-1]
                and macd.iloc[-2] < signal.iloc[-2]
                and current_volume > average_volume
            ):
                order_target(asset, -(cash // current_price))  # Short position
                sell_signal = True
            elif (
                current_price < lower_band
                and macd.iloc[-1] < signal.iloc[-1]
                and macd.iloc[-2] > signal.iloc[-2]
                and current_volume > average_volume
            ):
                order_target(asset, cash // current_price)  # Long position
                buy_signal = True
            else:
                order_target(asset, 0)  # Close position
        
        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band, macd=macd.iloc[-1], signal=signal.iloc[-1], volume=current_volume, average_volume=average_volume)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal=sell_signal,
            buy_signal=buy_signal,
            price_hist=prices,
            volume_hist=volumes
        )


In [ ]:
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)

## Added Volatility Filter

In [ ]:
def initialize(context):
    context.i = 0
    set_benchmark(symbol(symb))
    sids = bundle_data.asset_finder.sids
    context.assets = bundle_data.asset_finder.retrieve_all(sids)
    context.bollinger_window = 25  # Bollinger Bands window
    context.bollinger_dev = 1.5  # Number of standard deviations for Bollinger Bands
    context.macd_short_window = 8
    context.macd_long_window = 26
    context.macd_signal_window = 7
    context.max_volatility = 0.03
    
def handle_data(context, data):
    buy_signal = False
    sell_signal = False
    
    for asset in context.assets:
        # Get historical price data
        prices = data.history(asset, 'close', context.bollinger_window + context.macd_long_window + context.macd_signal_window + 1, frequency='1d')
        
        # Calculate Bollinger Bands
        sma = prices[-context.bollinger_window:].mean()
        rolling_std = prices[-context.bollinger_window:].std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Calculate MACD
        close_prices = prices[-context.macd_long_window:]
        macd, signal = calculate_macd(close_prices, context.macd_short_window, context.macd_long_window, context.macd_signal_window)

        # Calculate historical returns for volatility calculation
        returns = prices.pct_change().dropna()

        # Calculate volatility as the standard deviation of returns
        volatility = returns.std()

        # Get the current price
        current_price = data.current(asset, 'close')
    
        open_orders = get_open_orders()
        # Check if the asset is still tradable and if volatility is below a certain threshold
        if data.can_trade(asset) and volatility < context.max_volatility:
            cash = context.portfolio.cash
            # Generate signals based on Bollinger Bands and MACD
            if current_price > upper_band and macd.iloc[-1] > signal.iloc[-1] and macd.iloc[-2] < signal.iloc[-2]:
                order_target(asset, -(cash // current_price))  # Short position
                sell_signal = True
            elif current_price < lower_band and macd.iloc[-1] < signal.iloc[-1] and macd.iloc[-2] > signal.iloc[-2]:
                order_target(asset, cash // current_price)  # Long position
                buy_signal = True
            else:
                order_target(asset, 0)  # Close position
        
        # Record the values for later analysis
        record(price=current_price, upper=upper_band, lower=lower_band, macd=macd.iloc[-1], signal=signal.iloc[-1], volatility=volatility)
        record(
            AAPL=current_price,
            lower_band=lower_band, upper_band=upper_band,
            sell_signal=sell_signal,
            buy_signal=buy_signal,
            price_hist=prices,
            volatility=volatility
        )


In [ ]:
result = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=capital_base,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='multi_asset_snp100',
)

In [ ]:
result["vols"] = result.starting_cash.pct_change().rolling(30).std()
def my_rolling_sharpe(ser):
    return np.sqrt(90) * (ser.mean()/ser.std())
result["sharpe_ratio"] = result['starting_cash'].pct_change().rolling(235).apply(my_rolling_sharpe)
fig, ax = plt.subplots(1, 3, figsize=(21,6))
result['sharpe_ratio'].plot(ax = ax[0], title='Sharpe Ratio')
result['vols'].plot(ax = ax[1], title='Volatility')
result['ending_cash'].plot(ax = ax[2], title='Cash')
plt.show()

In [ ]:
pf.create_returns_tear_sheet(returns=result.algorithm_period_return, benchmark_rets=None)